In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd ./gdrive/MyDrive/DOD\ Dataset

In [ ]:
!pip install transformers
!pip install sentence-transformers
!pip install datasets
!pip install evaluate

In [ ]:
from torch.utils.data import Dataset, random_split
from pathlib import Path
from pandas import read_csv, concat
import numpy as np



class ClusterDataset(Dataset):
    def __init__(self, data_dir: str):
        data_dir = Path(data_dir)

        type_df = read_csv(data_dir / "cluster_dataset.csv")
        self.len = len(type_df)
        self.examples = type_df
        labels = [x.label for x in self.examples.iloc]
        labels = set(labels)
        labels = list(labels)
        print("These are the labels: ", labels)
        self.m = {}
        for i, l in enumerate(labels):
            self.m[l] = i


    def __getitem__(self, i):

        label = self.examples.iloc[i].label
        label = self.m[label]

        return {
            "text": self.examples.iloc[i].text,
            "label": label,
        }

    def __len__(self):
        return len(self.examples)

In [ ]:
dataset = ClusterDataset('./')

In [ ]:
len(dataset)

In [ ]:
dataset[5000]

In [ ]:
plt.savefig('cluster.png')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = [len(organize), len(prepare), len(authorities), len(national_federal), len(anticipate), len(enable)]
x_labels = ['Organize', 'Prepare', 'Authorities', 'National/Federal', 'Anticipate', 'Enable']

fig, ax = plt.subplots(figsize=(9, 7))
# fig, ax = plt.subplots()
ax.bar(x_labels, data, color='gray')

ax.set_title('Bar Chart')
# ax.set_xlabel('X Axis Label')
# ax.set_ylabel('Y Axis Label')

plt.show()

In [ ]:
organize = [x for x in dataset if x['label'] == 0]
prepare = [x for x in dataset if x['label'] == 1]
authorities = [x for x in dataset if x['label'] == 2]
national_federal = [x for x in dataset if x['label'] == 3]
anticipate = [x for x in dataset if x['label'] == 4]
enable = [x for x in dataset if x['label'] == 5]

In [ ]:
print(len(organize), len(prepare), len(authorities), len(national_federal), len(anticipate), len(enable))

In [ ]:
m_datasets = {}
m_datasets[0] = organize
m_datasets[1] = prepare
m_datasets[2] = authorities
m_datasets[3] = national_federal
m_datasets[4] = anticipate
m_datasets[5] = enable

In [ ]:
import random

train_dataset = []
val_dataset = []
test_dataset = []


def getRandomFromData(data, percent):
    size = int((percent * len(data)) / 100)

    output = []
    for i in range(size):
        element = random.choice(data)
        output.append(element)
        data.remove(element)

    return output, data

num_types = len(m_datasets.keys())

for i in range(num_types):
    dat = m_datasets[i]
    train_d, dat = getRandomFromData(dat, 60)
    val_d, dat = getRandomFromData(dat, 15)
    test_d, dat = getRandomFromData(dat, 25)

    train_dataset += train_d
    val_dataset += val_d
    test_dataset += test_d

In [ ]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

In [ ]:
organize = [x for x in test_dataset if x['label'] == 0]
prepare = [x for x in test_dataset if x['label'] == 1]
authorities = [x for x in test_dataset if x['label'] == 2]
national_federal = [x for x in test_dataset if x['label'] == 3]
anticipate = [x for x in test_dataset if x['label'] == 4]
enable = [x for x in test_dataset if x['label'] == 5]

In [ ]:
print(len(organize), len(prepare), len(authorities), len(national_federal), len(anticipate), len(enable))

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

# mydataset = Dataset.from_generator(dataset)

myTraindataset = Dataset.from_list(train_dataset)
myValdataset = Dataset.from_list(val_dataset)
myTestdataset = Dataset.from_list(test_dataset)


mydataset = DatasetDict({"train": myTraindataset, "validation": myValdataset, "test":myTestdataset})

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

#raw_datasets = load_dataset("glue", "mrpc")

# facebook/bart-base
#"bert-base-uncased"


checkpoint = "mukund/privbert"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = mydataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
mydataset

In [ ]:
mydataset['train'][0]

In [ ]:
from transformers import TrainingArguments, Trainer, logging

# training_args = TrainingArguments("test-trainer")

default_args = {
    "output_dir": "./",
    "evaluation_strategy": "epoch",
    "num_train_epochs": 6,
    # "log_level": "error",
    "logging_steps": 1,
    "log_level" : "info",
    "report_to": "none",
}

training_args = TrainingArguments(per_device_train_batch_size=64,
                                  # gradient_accumulation_steps=32,
                                  gradient_checkpointing=True,
                                  # optim="adafactor",
                                  **default_args,
                                )

In [ ]:
training_args

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(m_datasets.keys()))

In [ ]:
from datasets import load_metric
from sklearn.metrics import mean_squared_error
import evaluate
import numpy as np


accuracy = evaluate.load("accuracy")

# imdb = load_dataset("imdb")

f1 = evaluate.load("f1")

# imdb = load_dataset("imdb")


#F1
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    microF1 = f1.compute(predictions=predictions, references=labels, average='micro')
    macroF1 = f1.compute(predictions=predictions, references=labels, average='macro')
    weightF1 = f1.compute(predictions=predictions, references=labels, average='weighted')

    return {"micro-F1": microF1, "macroF1": macroF1, "weightF1": weightF1}


# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer, TrainerCallback
import torch


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    # callbacks=[GPUCallback()]
)

In [ ]:
!nvidia-smi

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
from transformers import Trainer, TrainerCallback
import torch


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    # callbacks=[GPUCallback()]
)

In [ ]:
trainer.evaluate()